<a href="https://colab.research.google.com/github/SabihMajid/langchain-module-0/blob/main/12_langchain_ecosystem/langgraph/course-notebooks/module-0/basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Academy

Welcome to LangChain Academy!

## Context

At LangChain, we aim to make it easy to build LLM applications. One type of LLM application you can build is an agent. There’s a lot of excitement around building agents because they can automate a wide range of tasks that were previously impossible.

In practice though, it is incredibly difficult to build systems that reliably execute on these tasks. As we’ve worked with our users to put agents into production, we’ve learned that more control is often necessary. You might need an agent to always call a specific tool first or use different prompts based on its state.

To tackle this problem, we’ve built [LangGraph](https://langchain-ai.github.io/langgraph/) — a framework for building agent and multi-agent applications. Separate from the LangChain package, LangGraph’s core design philosophy is to help developers add better precision and control into agent workflows, suitable for the complexity of real-world systems.

## Course Structure

The course is structured as a set of modules, with each module focused on a particular theme related to LangGraph. You will see a folder for each module, which contains a series of notebooks. A video will accompany each notebook to help walk through the concepts, but the notebooks are also stand-alone, meaning that they contain explanations and can be viewed independent of the videos. Each module folder also contains a `studio` folder, which contains a set of graphs that can be loaded into [LangGraph Studio](https://github.com/langchain-ai/langgraph-studio), our IDE for building LangGraph applications.

## Setup

Before you begin, please follow the instructions in the `README` to create an environment and install dependencies.

## Chat models

In this course, we'll be using [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which do a few things take a sequence of messages as inputs and return chat messages as outputs. LangChain does not host any Chat Models, rather we rely on third party integrations. [Here](https://python.langchain.com/v0.2/docs/integrations/chat/) is a list of 3rd party chat model integrations within LangChain! By default, the course with use [GEMINI_API_KEY](https://ai.google.dev/gemini-api/docs/api-key/) because it is both popular and performant. As noted, please ensure that you have an `GEMINI_API_KEY`.

Let's check that your `GEMINI_API_KEY` is set and, if not, you will be asked to enter it.

In [12]:
%%capture --no-stderr
!pip install -q TavilySearchResults
!pip install -q langchain_google_genai langchain_core langchain_community tavily_python

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0
)


In [5]:
result  = llm.invoke("where am I?")
result

AIMessage(content='I do not know where you are. I have no access to your location.  To find out where you are, you can use a map application on your phone or computer, or look for street signs or other landmarks.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-3050c2d9-171c-423f-8626-15e8dfce2136-0', usage_metadata={'input_tokens': 5, 'output_tokens': 46, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}})

Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: stream back chunks of the response
* `invoke`: call the chain on an input

And, as mentioned, chat models take [messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [10]:
from langchain_core.messages import HumanMessage, AIMessage

# Create a message
# Message list
messages = [
    HumanMessage(content="Hi", name="ALEX"),
    AIMessage(content='Hi there! How can I help you today?', name="AI ASSISTANT"),
    HumanMessage(content="what is python?", name="ALEX"),
    AIMessage(content="Python is a high-level, general-purpose programming language.Python is used in web game development, and much more.", name="AI ASSISTNT"),
    HumanMessage(content="how can i use this for llm app?", name="ALEX"),
    ]

# Invoke the model with a list of messages
llm.invoke(messages)

AIMessage(content='Python is a fantastic choice for building LLM applications.  It offers a rich ecosystem of libraries specifically designed for working with large language models. Here\'s how you can use it:\n\n**1. Choosing an LLM API or Library:**\n\n* **OpenAI API:** This is a popular choice, offering access to powerful models like GPT-3, GPT-4, and others.  You\'ll use the `openai` Python library to interact with the API.  This involves making API calls to generate text, translate languages, write different kinds of creative content, and answer your questions in an informative way.\n\n* **Hugging Face Transformers:** This library provides access to a vast collection of pre-trained language models, including many open-source alternatives to OpenAI\'s models.  It allows you to download and run models locally or use them via inference APIs.  This gives you more control and potentially lower costs, but requires more technical expertise to manage.\n\n* **Other APIs:**  There are many 

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


The interface is consistent across all chat models and models are typically initialized once at the start up each notebooks.

So, you can easily switch between models without changing the downstream code if you have strong preference for another provider.


## Search Tools

You'll also see [Tavily](https://tavily.com/) in the README, which is a search engine optimized for LLMs and RAG, aimed at efficient, quick, and persistent search results. As mentioned, it's easy to sign up and offers a generous free tier. Some lessons (in Module 4) will use Tavily by default but, of course, other search tools can be used if you want to modify the code for yourself.

In [30]:
import os
from google.colab import userdata
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

from langchain_community.tools.tavily_search import TavilySearchResults

os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
tool = TavilySearchResults(max_results=2)

tavily_search = TavilySearchResults(max_results=3, api_key=TAVILY_API_KEY)
search_docs= tavily_search.invoke("Where am i?")


In [31]:
search_docs

[{'url': 'https://www.where-am-i.co/',
  'content': 'My Location Now\nWhere am I - Map\nAllow current location access to proceed\nMy Location Now\nYou can share your current location by sending the following "where am I" link. Our Where am I app shows your current location on the map below or Google Maps and helps you find your coordinates and your location address.\n If you are using an Android device, here is the link to make sure your browser has been granted access to your location: Official Google support.\n If you are visiting this page on an iPhone or on iPad, make sure your browser has been granted access to your location: Official Apple support.\n Where am i right now tips: for privacy reasons, please use a secure connection to show\nmy current location on Google Maps (make sure the URL starts with https://).'},
 {'url': 'https://where-am-i.org/',
  'content': 'The where am I latitude and longitude tool will find the lat long of your current location in case you need for your 